In [ ]:
from datamap import experiment_files
experiment_files.keys()

In [1]:
import torch
print(torch.__version__)

1.12.1


In [3]:
from RODAN.basecall import load_model
from types import SimpleNamespace
import torch
#TODO fix module importing without hacking imports
#TODO vocab ATCG - but rna is AUCG
torchdict = torch.load('./RODAN/rna.torch', map_location="cpu")
origconfig = torchdict["config"]
d = origconfig
n = SimpleNamespace(**d)
args = {
    'debug':False, #False
    'arch':None,
}
print(n)
# print(type(origconfig))
model, device = load_model('./RODAN/rna.torch', config=n, args=SimpleNamespace(**args))
# model

namespace(activation='mish', amp=False, arch='[[-1, 256, 0, 3, 1, 1, 0], [-1, 256, 1, 10, 1, 1, 1], [-1, 256, 1, 10, 10, 1, 1], [-1, 320, 1, 10, 1, 1, 1], [-1, 384, 1, 15, 1, 1, 1], [-1, 448, 1, 20, 1, 1, 1], [-1, 512, 1, 25, 1, 1, 1], [-1, 512, 1, 30, 1, 1, 1], [-1, 512, 1, 35, 1, 1, 1], [-1, 512, 1, 40, 1, 1, 1], [-1, 512, 1, 45, 1, 1, 1], [-1, 512, 1, 50, 1, 1, 1], [-1, 768, 1, 55, 1, 1, 1], [-1, 768, 1, 60, 1, 1, 1], [-1, 768, 1, 65, 1, 1, 1], [-1, 768, 1, 70, 1, 1, 1], [-1, 768, 1, 75, 1, 1, 1], [-1, 768, 1, 80, 1, 1, 1], [-1, 768, 1, 85, 1, 1, 1], [-1, 768, 1, 90, 1, 1, 1], [-1, 768, 1, 95, 1, 1, 1], [-1, 768, 1, 100, 1, 1, 1]]\n', batchsize=30, dropout=0.1, epochs=30, gradclip=0, half=False, lr=0.002, name='1210', optimizer='ranger', orig={'name': '1210', 'seqlen': 4096, 'epochs': 30, 'optimizer': 'ranger', 'lr': 0.002, 'weightdecay': 0.01, 'batchsize': 30, 'dropout': 0.1, 'activation': 'mish', 'sqex_activation': 'mish', 'sqex_reduction': 32, 'trainfile': '/s/neptune/a/tmp/newrn

In [4]:
model(torch.rand(8,1,4096).to('cuda')).size()

torch.Size([420, 8, 5])

In [2]:
from rodan_pretrained import RodanPretrained
import torch
model = RodanPretrained().cuda()
model(torch.rand(4,1,4096).to('cuda'))

tensor([[0.3219],
        [0.3033],
        [0.3036],
        [0.3361]], device='cuda:0')

In [ ]:
from torchmetrics import ConfusionMatrix
import torch
target = torch.tensor([1, 1, 1,0, 0,0,0])
preds = torch.tensor([0, 1,0,1,0, 0,0])
confmat = ConfusionMatrix(num_classes=2, normalize='true')
confmat(preds, target)


In [ ]:
from datamap import experiment_files
len(list(experiment_files['pos_2022_fast'][0].iterdir()))

In [ ]:
from dataloading2 import nanopore_datamodule
dm = nanopore_datamodule(pos_files = 'pos_2022_fast', neg_files='pos_2022_fast', verbose=True)

In [ ]:
import pytorch_lightning as pl
from torch import nn
import torch
from torch.nn import functional as F
from torch.nn import RNN
import torchmetrics
from resnet1d.resnet1d import ResNet1D




In [ ]:
from transformers import BertModel, BertConfig
BertModel(BertConfig())

In [ ]:
from tst import Transformer
from transformer.src.benchmark import LSTM, BiGRU, ConvGru, FullyConv, FFN
import torch
# LSTM(input_dim=1, hidden_dim=64, output_dim=1, num_layers=3)
# BiGRU(input_dim=1, hidden_dim=64, output_dim=1, num_layers=3)
# ConvGru(input_dim=1, hidden_dim=64, output_dim=1, num_layers=3)
# FullyConv(input_dim=1, hidden_dim=64, output_dim=1)
# FFN(input_dim=1, hidden_dim=64, output_dim=1, num_layers=3)

#TODO chunk=none??
#TODO attention size
leng = 1000
model = Transformer(d_input=1, d_model=1, d_output=1,input_length=1000, q=8,v=8,h=8, N=4, attention_size=50, chunk_mode=None, pe='regular', skip_embedding=True)
# print(model)
print(model(torch.rand(32,1000,1)).size())
print(model)

In [ ]:
class GenericModel(pl.LightningModule):
    def __init__(self, learning_rate):
        super().__init__()
        self.learning_rate = learning_rate


        self.net = LSTM(input_dim=1, hidden_dim=64, output_dim=1, num_layers=3)
        # BiGRU(input_dim=1, hidden_dim=64, output_dim=1, num_layers=3)
        # ConvGru(input_dim=1, hidden_dim=64, output_dim=1, num_layers=3)
        # FullyConv(input_dim=1, hidden_dim=64, output_dim=1)
        # FFN(input_dim=1, hidden_dim=64, output_dim=1, num_layers=3)

        self.acc = torchmetrics.Accuracy()

    def forward(self, x):
        out = self.net.forward(x)
        return out[:,-1,:]
  
      

    def configure_optimizers(self):
      # print("LEARNING RATE:",self.learning_rate)
      optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate, weight_decay=0.01) #wd 0.01
      return optimizer

    def training_step(self, train_batch, batch_idx):
      x,y = train_batch
      output = self(x)
      loss = F.binary_cross_entropy_with_logits(output, y)
      self.log('train_loss', loss)
      acc =self.acc(output, y.int())
      self.log('train acc', acc)
      return loss

    def validation_step(self, val_batch, batch_idx):
      x,y = val_batch
      output = self(x)
      loss = F.binary_cross_entropy_with_logits(output, y)
      self.log('valid_loss', loss)
      acc = self.acc(output, y.int())
      self.log('valid acc', acc)

rnn = GenericModel(0.001)
rnn(torch.swapaxes(torch.rand(32,1,1000),-1,-2)).size()
